<a href="https://colab.research.google.com/github/tutsilianna/Gift_Recommendation_System/blob/main/Gift_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Аннотация

Список литературы:
1. https://education.yandex.ru/handbook/ml/article/intro-recsys
2. Анализ текстовых данных https://habr.com/ru/companies/otus/articles/774498/
---
https://neurohive.io/ru/tutorial/sozdaem-rekomendatelnuju-sistemu-s-ispolzovaniem-biblioteki-fastai/

https://habr.com/ru/companies/skillfactory/articles/585182/   

https://neerc.ifmo.ru/wiki/index.php?title=%D0%A0%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D1%8B

https://vc.ru/u/1389654-machine-learning/592278-5-luchshih-generatorov-sinteticheskih-dannyh-na-python-i-kak-ih-ispolzovat-kogda-vam-ne-hvataet-dannyh

### Есть три варианта реализации:


1.  Content-Based Filtering (Фильтрация на основе контента)

    **Принцип:** Анализирует характеристики подарков и предпочтения пользователя (пол, возраст, интересы, хобби).
    
    **Данные:** Описания подарков, профили пользователей.
    
    **Алгоритмы:**TF-IDF, cosine similarity.
    
    **Пример:** Если пользователь увлекается фотографией, система порекомендует фотоаппарат, объектив или аксессуары для фотосъемки.

2. Collaborative Filtering (Коллаборативная фильтрация):

    **Принцип:** Анализирует историю покупок и оценок пользователей, чтобы найти похожих пользователей и рекомендовать подарки, которые понравились им.

    **Данные:** История покупок, рейтинги, отзывы.

    **Алгоритмы:** K-Nearest Neighbors, Matrix Factorization.

    **Пример:** Если пользователь А и пользователь Б имеют похожую историю покупок, и пользователь А купил книгу, которая понравилась пользователю Б, система порекомендует эту книгу другим пользователям с похожими предпочтениями.



3. Hybrid Approach (Гибридный подход):

    **Принцип:** Комбинирует content-based и collaborative filtering для повышения точности рекомендаций.

    **Пример:** Система использует content-based filtering, чтобы найти подарки, соответствующие интересам пользователя, а затем использует collaborative filtering, чтобы ранжировать эти подарки на основе предпочтений похожих пользователей.



* Дополнительные факторы:

    **Бюджет пользователя:** Система должна учитывать бюджет пользователя при рекомендации подарков.

    **Повод:** Подарки на день рождения, Новый год или свадьбу будут отличаться.

    **Социальная информация:** Информация о друзьях и семье пользователя в социальных сетях может помочь в подборе подарков.

**Входные параметры:** (опционально) пол, возраст, интересы/хобби - строковое описание увлечений человека.

**Данные для обучения:** даные о пользователе (входные параметры) + какие подарки он выбрал. Эти данные будут синтетическими

## Генерация анкет

In [ ]:
import random

# Список интересов
interests_list = ["спорт", "музыка", "кино", "книги", "игры", "кулинария",
                  "путешествия", "искусство", "технологии", "мода", "красота",
                  "шоппинг", "автомобили", "рыбалка", "охота", "финансы",
                  "образование", "здоровье", "саморазвитие", "домашние животные",
                  "садоводство", "фотография", "танцы", "йога", "волонтерство",
                  "рисование", "живопись", "скульптура", "фотография", "музыка",
                  "танцы", "театр", "кино", "писательство", "поэзия", "рукоделие",
                  "дизайн", "кулинария", "марки", "монеты", "комиксы", "антиквариат",
                  "игрушки", "видеоигры", "компьютерные игры", "мобильные игры",
                  "настольные игры", "азартные игры"]

# Функция для генерации данных пользователя
def generate_user():
    gender = random.choice(["male", "female"])
    age = random.randint(18, 75)
    num_interests = random.randint(2, 5)  # Количество интересов
    interests = random.sample(interests_list, num_interests)
    return f"{gender}\t{age}\t{','.join(interests)}"

# Генерация и запись данных
with open("users.txt", "w") as f:
    for _ in range(5000):
        user_data = generate_user()
        f.write(user_data + "\n")

print("Файл users.txt создан!")

Файл users.txt создан!


## Подключение репозитория

In [ ]:
! git clone https://github.com/tutsilianna/Gift_Recommendation_System

Cloning into 'Gift_Recommendation_System'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 52 (delta 16), reused 24 (delta 5), pack-reused 0
Receiving objects: 100% (52/52), 15.59 MiB | 11.75 MiB/s, done.
Resolving deltas: 100% (16/16), done.


## Импорт необходимых библиотек

In [ ]:
import re
import nltk
import json
import warnings
import pandas as pd
import numpy as np
from nltk import stem
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from gensim.models import Word2Vec
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download("stopwords")
warnings.filterwarnings("ignore")

mystem = Mystem()
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Чтение данных и предобработка

In [ ]:
data = pd.read_csv('/content/Gift_Recommendation_System/data/raw/ozon_data_.csv')
data.head(2)

,название,описание,код товара,оценка,количество отзывов,цена,категория,подкатегория,подподкатегория,ссылка
0,Проектор для смартфона 2-го поколения своими р...,Проектор для смартфона (проектор для смартфона...,Артикул: 1320591182,3.3,13,808,Электроника,Телевизоры и видеотехника,Проекторы,https://www.ozon.ru/product/proektor-dlya-smar...
1,"Проектор 90411567, белый",3D увеличитель экрана телефона – это универсал...,Артикул: 619939085,4.4,205,300,Электроника,Телевизоры и видеотехника,Проекторы,https://www.ozon.ru/product/proektor-90411567-...


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22591 entries, 0 to 22590
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   название            22591 non-null  object 
 1   описание            16366 non-null  object 
 2   код товара          21916 non-null  object 
 3   оценка              22591 non-null  float64
 4   количество отзывов  22591 non-null  int64  
 5   цена                22591 non-null  int64  
 6   категория           22591 non-null  object 
 7   подкатегория        22591 non-null  object 
 8   подподкатегория     20793 non-null  object 
 9   ссылка              22591 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 1.7+ MB


## **1. Подготовка данных**



### **Предобработка данных:** обработка пропущенных значений, удаление дубликатов, преобразование столбца "код товара";    


In [ ]:
data['подподкатегория'] = data['подподкатегория'].fillna('')
data['код товара'] = data['код товара'].fillna('Артикул: 0')
data['описание'] = data['описание'].fillna('')

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
# удаление слова из "Код товара", удаление строк,
# где нет атрибута и преобразование столбца к численному типу

data['код товара'] = data['код товара'].str.split().str.get(-1)
data = data.loc[data['код товара'].str.isdigit()]
data['код товара'] = data['код товара'].astype(np.int64)

In [ ]:
data.drop_duplicates(subset=['код товара'], keep='first', inplace=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18427 entries, 0 to 22589
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   название            18427 non-null  object 
 1   описание            18427 non-null  object 
 2   код товара          18427 non-null  int64  
 3   оценка              18427 non-null  float64
 4   количество отзывов  18427 non-null  int64  
 5   цена                18427 non-null  int64  
 6   категория           18427 non-null  object 
 7   подкатегория        18427 non-null  object 
 8   подподкатегория     18427 non-null  object 
 9   ссылка              18427 non-null  object 
dtypes: float64(1), int64(3), object(6)
memory usage: 1.5+ MB


### **Составление набора данных в виде анкет:** с помощью GPT4 была получена переменная `test`, в которой каждому интересу из переменной `interests_list` была сопоставлена подподкатегория из переменной `categories_list`


In [ ]:
# Список интересов
interests_list = ["спорт", "музыка", "кино", "книги", "игры", "кулинария",
                  "путешествия", "искусство", "технологии", "мода", "красота",
                  "шоппинг", "автомобили", "рыбалка", "охота", "финансы",
                  "образование", "здоровье", "саморазвитие", "домашние животные",
                  "садоводство", "фотография", "танцы", "йога", "волонтерство",
                  "рисование", "живопись", "скульптура",
                  "театр", "писательство", "поэзия", "рукоделие",
                  "дизайн", "марки", "монеты", "комиксы", "антиквариат",
                  "игрушки", "видеоигры", "компьютерные игры", "мобильные игры",
                  "настольные игры", "азартные игры"]

categories_list = [category for category in set(data['подподкатегория'].values) if category != '']
categories = [f'{x[0]}/{x[1]}/{x[2]}' for x in data.groupby(['категория', 'подкатегория'])['подподкатегория'].value_counts().index]
print(f'Количество интересов: {len(interests_list)},\nКоличество категорий вариант №1: {len(categories_list)},\nКоличество категорий вариант №2: {len(categories)}')

Количество интересов: 43,
Количество категорий вариант №1: 277,
Количество категорий вариант №2: 331


` interests_categories = { "интерес": ['подподкатегория', ..],  ... }`

In [ ]:
interests_categories = {
    "спорт":             ["Красота, здоровье и спорт", "Cпортивные игры"],

    "музыка":            ["Искусство и культура", "Музыкальные инструменты", "Уход за музыкальными инструментами",
                          "Гитары и аксессуары", "Струнные инструменты", "Ударные и перкуссия", "Клавишные инструменты"],

    "кино":              ["Искусство и культура", "Кино", 'Проекторы', 'Экраны и проекторы'],

    "книги":             ["Книги", "Художественная литература", "Нехудожественная литература", "Проза", "Поэзия",
                          "Сентиментальная проза", "Религиозные книги", "Биографии и истории успеха", "Юридическая литература",
                          "Саморазвитие", "Компьютерные технологии", "Итальянский язык", "Наборы", "Другие языки", "Энциклопедии и справочники",
                          "Медицина", "История", "Фантастика", "Публицистика", "Английский язык", "Эзотерика и спиритизм", "Французский язык",
                          "Путешествия. Путеводители", "Учебная литература", "Пьесы и драматургия", "Товары для рукоделия",
                          "Научно-популярная литература", "Фольклор", "Молодежная и подростковая литература", "Немецкий язык",
                          "Античная литература", "Научная литература", "Детям и родителям", "Инвестиции и недвижимость",
                          "Финансы", "Манга", 'Детективы', "Фэнтези", "Приключения", "Ужасы и триллеры"],

    "игры":              ["Игры", "Игровые приставки", "Игры для Nintendo Switch", "Игры для Xbox Series", "Игры для Xbox 360",
                          "Игры для PlayStation Portable/Vita", "Игры для Xbox One", "Игры для всей семьи (Family)",
                          "Игры для Nintendo Wii/3DS", "Игры для PlayStation 4", "Игры для PlayStation 5",
                          "Игры для PlayStation 2/3", "Настольные игры", "Дополнения к настольным играм",
                          "Ходилки",  "Робототехника", 'Боевики', 'Геймпады и джойстики', 'Гонки (Racing)', 'Детективы', 'Игровые адаптеры',
                          "Очки виртуальной реальности для смартфонов",  "Видеоигры", "Компьютерные игры",
                          "Мобильные игры", "Настольные игры", "Азартные игры", "Приключения (Adventure)", "Игровые консоли",
                          "Аркады (Arcade)", "Ролевые игры (RPG & MMORPG)", "Стратегии (Strategy)", "Дополнения", "Лото",
                          "Сюжетно-ролевые игрушки", "Симуляторы (Simulation)", "Модели и фигурки",
                          "Ужасы (Horror)", "Драки (Fighting)", "Шутеры и экшн (Shooter & Action)", "Квесты и логика (Quest & Logic)",
                          "Интерактивные и электронные игрушки", "Военная техника и оружие", "Фигурки и аксессуары", "Гонки",
                          "Фэнтези", "Приключения", "Ужасы и триллеры","Спортивные игры (Sports)"],

    "кулинария":         ["Кулинария", "Кулинарные книги", "Домоводство и хобби", "Тарелки и блюда", "Предметы сервировки", "Стекло и всё для бара"],

    "путешествия":       ["Путешествия. Путеводители", "Сувениры и подарки"],

    "искусство":         ["Искусство и культура", "Материалы и инструменты для ювелиров", "Пигменты",
                          "Рисование жидким акрилом (Флюид-арт)", "Наборы для лепки", "Мольберты и этюдники",
                          "Штампы и трафареты", "Картины из пайеток", "Картины из эпоксидной смолы",
                          "Авторская посуда", "Средства для очищения кистей", "Топперы для торта",
                          "Вазы, кашпо", "Выжигание и поделки из дерева", "Создание украшений",
                          "Пластилин", "Глина для лепки", "Реставрация и золочение", "Инструменты",
                          "Авторские игрушки", "Бумага для черчения и рисования",
                          "Наборы для квиллинга", "Шелкография", "Духовые инструменты",
                          "Скетчбуки", "Краски", "Уголь, сангина, сепия и соус", "Маркеры",
                          "Наборы для росписи", "Подставки", "Картриджи для 3D ручек", "Шитье",
                          "Поделки из бумаги", "Веники для бань", "Изготовление ножей", "Ткачество и прядение",
                          "Текстиль", "Рисование на воде Эбру", "Клеи и лак для полимерной глины", "Наперстки",
                          "Вязание", "Краски Холи", "Валяние", "Наборы для рисования песком", "Элементы интерьера",
                          "Свадебные товары", "Открытки", "Заготовки для декорирования", "Подсвечники и аромалампы",
                          "Наборы для создания фоторамок", "Декупаж", "Папье-маше", "Модели из дерева", "Кожевенное дело",
                          "Алмазные мозаики", "Лэмпворк", "Ароматы для дома", "Парфюмированный набор", "Карандаши",
                          "Цветной песок", "Значки", "Наборы для создания слепков", "Украшения на машину",
                          "Изготовление игрушек", "Пиньяты", "Пастель", "Глиттеры", "Флористика", "Досуг и творчество",
                          "Фотобутафория", "Картины и панно", "Аксессуары и материалы", "Куклы",  "Карточки",
                          "Наборы для гигиены", "Картины по номерам", "Воздушные шары", "Свечи и подсвечники",
                          "Антикварные газеты и журналы", "Пэчворк и квилтинг", "Раскраски", "Папертоли",
                          "Фотоаппараты мгновенной печати", "Билеты", "Наборы для создания причесок",
                          "Масса и тесто для лепки", "Наборы для создания духов", "Открытки и конверты",
                          "Цветы", "Наборы средств для лица", "Интерьерные миниатюры и сборка зданий",
                          "Косметические наборы", "Холсты, артборды и картон грунтованный", "Хранение коллекций",
                          "Аксессуары"],

    "технологии":        ["Компьютерные технологии", "Электроника", "Компьютеры",  "Периферия", "Игровые планшеты", 'Проекторы'],

    "мода":              ["Журналы и наклейки", "Мода", "Красота", "Шоппинг", "Этикетки и пробки", "Карнавальные товары",
                          "Подарочные наборы", "Марки", "Банкноты", "Журналы и газеты",
                          "Наборы для макияжа", "Испанский язык", "Английский язык", "Значки"],

    "красота":           ["Красота, здоровье и спорт", "Изготовление косметики", "Наборы косметики", "Косметические наборы",
                          "Наборы средств для лица", "Наборы для гигиены"],

    "шоппинг":           ["Всё для дома", "Подарочные наборы", "Сувениры и подарки"],

    "автомобили":        ["Автомобили и мотоциклы"],

    "рыбалка":           ["Инструменты и инвентарь", "Аксессуары"],

    "охота":             ["Инструменты и инвентарь", "Аксессуары"],

    "финансы":           ["Финансы", "Экономика", "Бухгалтерский учет и делопроизводство", "Инвестиции и недвижимость",
                          "Маркетинг и продажи", "Менеджмент и управление", "Банкноты", "Монеты"],

    "образование":       ["Бизнес-образование и развитие карьеры", "Развитие детей", "Учебная литература",
                          "Научно-популярная литература", "Детям и родителям", "Познавательная литература"],

    "здоровье":          ["Красота, здоровье и спорт", "Медицина", "Средства для ухода", "Наборы для гигиены"],

    "саморазвитие":      ["Саморазвитие", "Трансформационные психологические игры", "Психология"],

    "домашние животные": ["Домашние животные", "Товары для животных"],

    "садоводство":       ["Домоводство и хобби", "Инструменты и инвентарь", "Садоводство", "Цветы"],

    "фотография":        ["Фотография", "Цифровые фоторамки", "Фотоаппараты мгновенной печати"],

    "танцы":             ["Танцы"],
    "йога":              ["Красота, здоровье и спорт", "Йога"],

    "волонтерство":      ["Волонтерство"],

    "рисование":         ["Рисование жидким акрилом (Флюид-арт)", "Мольберты и этюдники", "Штампы и трафареты",
                          "Скетчбуки", "Краски", "Уголь, сангина, сепия и соус", "Маркеры", "Наборы для росписи",
                          "Картриджи для 3D ручек", "Рисование на воде Эбру", "Наборы для рисования песком",
                          "Карандаши", "Пастель", "Раскраски"],

    "живопись":          ["Рисование жидким акрилом (Флюид-арт)", "Мольберты и этюдники", "Штампы и трафареты",
                          "Картины из пайеток", "Картины из эпоксидной смолы", "Средства для очищения кистей",
                          "Краски", "Уголь, сангина, сепия и соус", "Маркеры", "Наборы для росписи",  "Картриджи для 3D ручек",
                          "Рисование на воде Эбру", "Карандаши", "Пастель", "Картины и панно", "Картины по номерам"],

    "скульптура":        ["Наборы для лепки", "Глина для лепки", "Пластилин", "Гипс", "Масса и тесто для лепки"],

    "театр":             ["Театр", "Пьесы и драматургия"],

    "писательство":      ["Письменные принадлежности", "Бумага для черчения и рисования", "Карандаши"],

    "поэзия":            ["Поэзия"],

    "рукоделие":         ["Гобелены и стринг-арт", "Вышивание", "Изготовление косметики", "Наборы для лепки",
                          "Вышивание", "Штампы и трафареты", "Картины из пайеток", "Авторская посуда",
                          "Топперы для торта", "Выжигание и поделки из дерева", "Создание украшений",
                          "Пластилин", "Глина для лепки", "Принадлежности", "Оригами", "Реставрация и золочение",
                          "Шелкография", "Скетчбуки", "Краски", "Уголь, сангина, сепия и соус", "Маркеры",
                          "Наборы для росписи", "Подставки", "Картриджи для 3D ручек", "Шитье", "Поделки из бумаги",
                          "Веники для бань", "Аксессуары и атрибутика для гаданий", "Изготовление ножей",
                          "Ткачество и прядение", "Текстиль", "Рисование на воде Эбру", "Клеи и лак для полимерной глины",
                          "Наперстки", "Вязание", "Краски Холи", "Валяние", "Наборы для рисования песком",
                          "Элементы интерьера", "Свадебные товары", "Открытки", "Заготовки для декорирования",
                          "Подсвечники и аромалампы", "Наборы для создания фоторамок", "Декупаж", "Папье-маше",
                          "Модели из дерева", "Кожевенное дело", "Алмазные мозаики", "Лэмпворк", "Наборы для макияжа",
                          "Ароматы для дома", "Парфюмированный набор", "Карандаши", "Цветной песок", "Значки",
                          "Наборы для создания слепков", "Украшения на машину", "Изготовление игрушек", "Пиньяты",
                          "Пастель", "Глиттеры", "Флористика", "Досуг и творчество", "Фотобутафория", "Картины и панно",
                          "Аксессуары и материалы", "Куклы",  "Карточки", "Товары для рукоделия", "Наборы для гигиены",
                          "Картины по номерам", "Воздушные шары", "Свечи и подсвечники", "Антикварные газеты и журналы",
                          "Пэчворк и квилтинг", "Наборы косметики", "Раскраски", "Папертоли", "Наборы для создания причесок",
                          "Масса и тесто для лепки", "Наборы для создания духов", "Открытки и конверты", "Цветы",
                          "Наборы средств для лица", "Интерьерные миниатюры и сборка зданий", "Косметические наборы",
                          "Холсты, артборды и картон грунтованный", "Скрапбукинг", "Мыловарение", "Наборы для рисования",
                          "Аппликации", "Плетение"],

    "дизайн":            ["Дизайн", "Мольберты и этюдники", "Штампы и трафареты", "Бумага для черчения и рисования",
                          "Карандаши", "Наборы для создания фоторамок", "3D ручки"],

    "марки":             ["Марки", "Коллекционирование", "Хранение коллекций"],

    "монеты":            ["Монеты", "Коллекционирование", "Хранение коллекций"],

    "комиксы":           ["Комиксы", "Журналы и наклейки"],

    "антиквариат":       ["Антиквариат", "Минералы и окаменелости", "Амулеты, талисманы и ловцы снов",
                          "Античная литература", "Автографы и фото коллекционные", "Награды и кубки",
                          "Банкноты", "Монеты", "Антикварные газеты и журналы"],

    "игрушки":           ["Мягкие игрушки", "Игрушечное оружие", "Роботы и трансформеры", "Развивающие игры",
                          "Игрушки для малышей", "Конструкторы", "Игрушки-антистресс", "Сюжетно-ролевые игрушки",
                          "Модели и фигурки", "Игрушки для ванной", "Интерактивные и электронные игрушки", "Куклы",
                          "Военная техника и оружие", "Фигурки и аксессуары", "Игрушечный транспорт",
                          "Куклы и аксессуары", "Мыльные пузыри", "Игрушки для ванной", "Игрушечный транспорт",
                          "Игральные карты", "Головоломки", "Трансформационные психологические игры",
                          "Корабли и подводные лодки", "Мягкие игрушки", "Игрушечное оружие","Радиоуправляемые игрушки",
                          "3D ручки", "Кинетический песок", "Роботы и трансформеры", "Развивающие игры", "Игрушки для малышей",
                          "Конструкторы", "Игрушки-антистресс", "Пазлы",'Самолеты и космические корабли', "Копилки", "Слаймы"],
    "видеоигры":         ["Видеоигры", "Игры для Nintendo Switch", "Игры для Xbox Series", "Игры для Xbox 360",
                          "Игры для PlayStation Portable/Vita", "Игры для Xbox One", "Игры для всей семьи (Family)",
                          "Игры для Nintendo Wii/3DS", "Игры для PlayStation 4", "Игры для PlayStation 5",
                          "Игры для PlayStation 2/3", 'Очки виртуальной реальноcти для смартфонов'],
    "компьютерные игры": ["Компьютерные игры", "Игры для Nintendo Switch", "Игры для Xbox Series",
                          "Игры для Xbox 360", "Игры для PlayStation Portable/Vita", "Игры для Xbox One",
                          "Игры для всей семьи (Family)", "Игры для Nintendo Wii/3DS", "Игры для PlayStation 4",
                          "Игры для PlayStation 5", "Игры для PlayStation 2/3"],

    "мобильные игры":    ["Мобильные игры"],

    "настольные игры":   ["Настольные игры", "Дополнения к настольным играм", "Ходилки", "Игральные карты",
                          "Головоломки", "Трансформационные психологические игры", "Корабли и подводные лодки",
                          "Дополнения", "Лото", "Карты Таро", "Руны",  "Сюжетно-ролевые игрушки", "Шахматы", "Шашки", "Нарды",
                          "Карточные игры"],

    "азартные игры":     ["Азартные игры", "Игральные карты", "Игры в рулетку", "Наборы для покера", "Карты оплаты", "Домино и маджонг"],
    "коллекционирование":['Банкноты', 'Наборы', 'Значки', 'Этикетки и пробки', 'Карточки',
                          'Куклы', 'Билеты', 'Автографы и фото коллекционные',
                          'Минералы и окаменелости', 'Открытки и конверты', 'Наперстки',
                          'Хранение коллекций', 'Журналы и наклейки', 'Монеты', 'Марки',
                          'Календари', 'Средства для ухода', 'Модели и фигурки', 'Насекомые', "Фигурки"]
}
# size = 18427
interests_codes = {}
for interest in interests_categories:
    codes = []
    for word in interests_categories[interest]:
        codes += data.loc[data['подподкатегория'] == word, 'код товара'].values.tolist()
    if codes:
        interests_codes[interest] = codes
        print(f'{interest} {interests_codes[interest]}')

with open('interests_categories.json', 'w', encoding='utf-8') as file:
    json.dump(interests_codes, file, indent=4,  ensure_ascii=False)

keys = list(interests_categories.keys())
interests_codes_code  = {keys[i]: i+1  for i in range(len(keys))}

спорт [2130524, 482231739, 250970185, 250803970, 231648553, 31432890, 252734105, 1202755130, 658045563, 231648510, 1262906846, 31763354, 291992349, 324401964, 157804021, 1431518928, 1252526404, 388014458, 231648399, 211432418, 1358455694, 207921218, 267303955, 783514028, 819878142, 660370742, 158873698, 533841531, 250456382, 211432837, 249025521, 250876411, 1352666135, 1397875334, 820918597, 249179759, 355950751, 249166924, 250949612, 901137625, 250059248, 26419023, 34870845, 934325701, 892379875, 267320951, 249173671, 137652886, 250999058, 253327264, 181292157, 1413807149, 1171570200, 181292155, 24700170, 805198893, 27936698, 147055608, 871459123, 135462206, 730856796, 1282867657, 571118709, 166482309, 796387608, 256442249, 135892589, 805035664, 298278173, 269215469, 1427919963, 1275935926, 1418775173, 747554009, 229445055, 383685217, 587975281, 798108521, 1253970847, 855605187, 801644242, 1116580961, 166567508, 1469262973, 909776699, 229445173, 1112990878, 260533998, 981621799, 80164

In [ ]:
import random

user_vectors = {}
for i in range(1000):
    user_vectors[i] = [] # интересы, коды товаров
    random.sample(interests_list, random.randint(2, 11))
    interests_list
    interests_codes

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Создание словаря для хранения векторов товаров
item_vectors = {}

# Функция для создания вектора товара
def create_item_vector(row):
    vector = []
    vector.append(row['оценка'])
    vector.append(row['количество отзывов'])
    vector.append(row['цена'])
    vector += [1 if cat == row['категория'] else 0 for cat in data['категория'].unique()]
    vector += [1 if subcat == row['подкатегория'] else 0 for subcat in data['подкатегория'].unique()]
    vector += [1 if subsubcat == row['подподкатегория'] else 0 for subsubcat in data['подподкатегория'].unique()]
    return vector

# Заполнение словаря векторами товаров
for index, row in data.iterrows():
    item_vectors[row['код товара']] = create_item_vector(row)

# Функция для получения рекомендаций
def get_recommendations(item_id, top_n=3):
    input_vector = item_vectors[item_id]
    similarities = {}
    for key, value in item_vectors.items():
        if key != item_id:
            similarities[key] = cosine_similarity([input_vector], [value])[0][0]
    return sorted(similarities, key=similarities.get, reverse=True)[:top_n]

# Пример использования
recommendations = get_recommendations(926943324)
print(f"Рекомендации для товара с кодом 926943324: {recommendations}")

# Вывод рекомендаций с информацией о товаре
data.loc[data['код товара'].isin(recommendations), ['название', 'описание', 'ссылка']]

Рекомендации для товара с кодом 926943324: [173139380, 161855043, 1036269459]


,название,описание,ссылка
3070,"Настольные классические игры ""Шахматы, шашки"" ...",,https://www.ozon.ru/product/nastolnye-klassich...
3812,"Подарочный набор DREAMBOX ""БРАТУ"" Подарок для ...",Условия хранения\n\nХранить в защищенном от пр...,https://www.ozon.ru/product/podarochnyy-nabor-...
14103,Декоративные Сухие Блестки для творчества и ру...,Комплектация\n\nДекоративные Сухие Блестки Lux...,https://www.ozon.ru/product/dekorativnye-suhie...


### **Предобработка текста:** лемматизация и стемминг для поля "текст", где "текст" = "название", "описание" и "ключевые слова" (объединение "категории", "подкатегории" и "подподкатегории"), чтобы привести слова к базовой форме и улучшить поиск.


In [ ]:
data['ключевые слова'] = data['категория'] + ', ' + data['подкатегория'] + ', ' + data['подкатегория']
data.drop(columns=['категория', 'подкатегория', 'подподкатегория'], axis=1, inplace=True)

In [ ]:
text_columns = [column for column in data.columns if data[column].dtype == 'object' and column != 'ссылка']
text_columns

['название', 'описание', 'ключевые слова']

In [ ]:
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]

    text = " ".join(tokens)

    return text

In [ ]:
data_ = data.copy(deep=True)

In [ ]:
data["текст"] = data[text_columns].apply(lambda x: ". ".join(x), axis=1)
data.head(2)

,название,описание,код товара,оценка,количество отзывов,цена,ссылка,ключевые слова,текст
0,Проектор для смартфона 2-го поколения своими р...,Проектор для смартфона (проектор для смартфона...,1320591182,3.3,13,808,https://www.ozon.ru/product/proektor-dlya-smar...,"Электроника, Телевизоры и видеотехника, Телеви...",Проектор для смартфона 2-го поколения своими р...
1,"Проектор 90411567, белый",3D увеличитель экрана телефона – это универсал...,619939085,4.4,205,300,https://www.ozon.ru/product/proektor-90411567-...,"Электроника, Телевизоры и видеотехника, Телеви...","Проектор 90411567, белый. 3D увеличитель экран..."


In [ ]:
data['текст'] = data['текст'].apply(preprocess_text)

In [ ]:
data[['текст']].head(3)

,текст
0,проектор смартфон 2 го поколение свой рука под...
1,проектор 90411567 белый 3d увеличитель экран т...
2,проектор 90411553 3d увеличительный экран ваш ...


### **Векторизация данных:** TF-IDF и word2vec для преобразования текстовых данных ("название", "описание" и "ключевые слова") в числовые векторы.


#### TD-IDF

In [ ]:
"""
# Создание TF-IDF векторайзера
vectorizer = TfidfVectorizer(max_features=10)  # Можно настроить max_features

# Обучение векторайзера и преобразование текста в векторы
tfidf_vectors = vectorizer.fit_transform(data["текст"])

# Преобразование в DataFrame
tfidf_df = pd.DataFrame(tfidf_vectors.toarray(), columns=vectorizer.get_feature_names_out())

# Объединение с исходным DataFrame
data_tfidf = pd.concat([data, tfidf_df], axis=1)

data_tfidf.head(2)
"""

'\n# Создание TF-IDF векторайзера\nvectorizer = TfidfVectorizer(max_features=10)  # Можно настроить max_features\n\n# Обучение векторайзера и преобразование текста в векторы\ntfidf_vectors = vectorizer.fit_transform(data["текст"])\n\n# Преобразование в DataFrame\ntfidf_df = pd.DataFrame(tfidf_vectors.toarray(), columns=vectorizer.get_feature_names_out())\n\n# Объединение с исходным DataFrame\ndata_tfidf = pd.concat([data, tfidf_df], axis=1)\n\ndata_tfidf.head(2)\n'

#### Word2Vec

In [ ]:
"""
# Подготовка текстов для Word2Vec
text_corpus = [text.split() for text in data["текст"]]

# Обучение Word2Vec модели
model = Word2Vec(text_corpus, vector_size=10, window=5, min_count=5, workers=4)  # Настройте параметры

# Получение векторов для каждого текста
word2vec_vectors = []
for text in text_corpus:
    word_vectors = [model.wv[word] for word in text if word in model.wv]
    if word_vectors:
        text_vector = np.mean(word_vectors, axis=0)
    else:
        text_vector = np.zeros(model.vector_size)
    word2vec_vectors.append(text_vector)

# Преобразование в DataFrame
word2vec_df = pd.DataFrame(word2vec_vectors)

# Объединение с исходным DataFrame
data_word2vec = pd.concat([data, word2vec_df], axis=1)

data_word2vec.head(2)
"""

'\n# Подготовка текстов для Word2Vec\ntext_corpus = [text.split() for text in data["текст"]]\n\n# Обучение Word2Vec модели\nmodel = Word2Vec(text_corpus, vector_size=10, window=5, min_count=5, workers=4)  # Настройте параметры\n\n# Получение векторов для каждого текста\nword2vec_vectors = []\nfor text in text_corpus:\n    word_vectors = [model.wv[word] for word in text if word in model.wv]\n    if word_vectors:\n        text_vector = np.mean(word_vectors, axis=0)\n    else:\n        text_vector = np.zeros(model.vector_size)\n    word2vec_vectors.append(text_vector)\n\n# Преобразование в DataFrame\nword2vec_df = pd.DataFrame(word2vec_vectors)\n\n# Объединение с исходным DataFrame\ndata_word2vec = pd.concat([data, word2vec_df], axis=1)\n\ndata_word2vec.head(2)\n'

### **Нормализация числовых данных:** Нормализовация данные о цене и количестве отзывов, чтобы они имели одинаковый масштаб.


In [ ]:
# scaler = MinMaxScaler().fit(data[normalize_column])

# data_normalize = scaler.transform(data[normalize_column])
# data[normalize_column] = pd.DataFrame(data_normalize, columns = normalize_column)

# data[normalize_column].head(2)

In [ ]:
# normalize_column = [column for column in data.columns if data[column].dtype != 'object' and column != 'код товара']
# data[normalize_column].head(2)

## **2.  Модель:**

* **Контент-базированная фильтрация:**
    * Вычислить сходство между векторами запроса пользователя и векторами товаров.
    * Для этого можно использовать косинусную меру или евклидово расстояние.
    * Рекомендуйте товары с наибольшим сходством.

Вычисление сходства между векторами запроса пользователя (кому, на какой праздник подарок), интересы и векторами товаров (название + описание + категории) с помощью  `linear_kernel`



In [ ]:
def filter_products(data, query, interests, price_range):
    # Пример фильтрации по ценовому диапазону
    filtered_data = data.loc[(data['цена'] >= price_range[0]) & (data['цена'] <= price_range[1])]

    # Пример фильтрации по интересам
    tfidf = TfidfVectorizer(stop_words=russian_stopwords)
    tfidf_matrix = tfidf.fit_transform(filtered_data['текст'])
    query = query + ' ' + ' '.join(interests)
    query_vec = tfidf.transform([query])

    filtered_data['similarity'] = linear_kernel(query_vec, tfidf_matrix).flatten()

    return filtered_data.sort_values(by=['similarity', 'оценка', 'количество отзывов'], ascending=False)[:10]

In [ ]:
# Пример запроса пользователя
query = 'подарок папе на день рождения 30 лет'
interests = ['машины', 'электроника']
price_range = (500, 2000)

# Получение рекомендаций
filter_products(data, query, interests, price_range)[['название', 'ключевые слова',  'цена', 'ссылка', 'код товара']]

,название,ключевые слова,цена,ссылка,код товара
235,Аккумуляторный блок,"Электроника, Аксессуары для электроники, Аксес...",905,https://www.ozon.ru/product/akkumulyatornyy-bl...,1505883105
271,Сделай сам Электронные шестеренки и механическ...,"Электроника, Аксессуары для электроники, Аксес...",1220,https://www.ozon.ru/product/sdelay-sam-elektro...,1391730221
254,Power Functions 8883/1 Средний мотор M-Motor с...,"Электроника, Аксессуары для электроники, Аксес...",1163,https://www.ozon.ru/product/power-functions-88...,1217816637
253,Дополнение к набору робототехники / Ресурсный ...,"Электроника, Аксессуары для электроники, Аксес...",1265,https://www.ozon.ru/product/dopolnenie-k-nabor...,646844950
265,"Шасси основа для мобильных роботов ардуино, ма...","Электроника, Аксессуары для электроники, Аксес...",1677,https://www.ozon.ru/product/shassi-osnova-dlya...,1142343926
236,WEDO2.0 Датчик наклона/45305/200+ чертежи,"Электроника, Аксессуары для электроники, Аксес...",782,https://www.ozon.ru/product/wedo2-0-datchik-na...,1313345590
233,WEDO2.0 Датчик перемещения/45304/200+ рисунков,"Электроника, Аксессуары для электроники, Аксес...",782,https://www.ozon.ru/product/wedo2-0-datchik-pe...,1313337750
239,Электродвигатель Электрический Средний линейны...,"Электроника, Аксессуары для электроники, Аксес...",861,https://www.ozon.ru/product/elektrodvigatel-el...,1479801517
3839,Подарочные наборы для женщин: подарок маме сес...,"Продукты питания, Выпечка и сладости, Выпечка ...",1439,https://www.ozon.ru/product/podarochnye-nabory...,1088337894
11008,"Линейка для квилтинга и пэчворка, 30 x 30 см, ...","Хобби и творчество, Рукоделие, Рукоделие",1119,https://www.ozon.ru/product/lineyka-dlya-kvilt...,1528410848


## **3.  Гибридный подход:**

* Комбинируйте контент-базированную и коллаборативную фильтрацию для большей точности рекомендаций.

Использование ruBERT


In [ ]:
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd


# Инициализация модели и токенизатора BERT
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased')

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Функция для получения эмбеддингов
def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.squeeze().detach().numpy()

# Получение эмбеддингов для описаний товаров
data[:100]['текст'] = data[:100]['текст'].apply(lambda x: get_embedding(x))

# Пример профиля пользователя на основе интересов
user_interests = 'фитнес йога здоровье'
user_profile = get_embedding(user_interests)

# Рекомендации на основе гибридного подхода
def recommend_hybrid(user_profile, item_embeddings, top_n=10):
    similarities = cosine_similarity(user_profile, item_embeddings)
    recommendations = similarities.argsort()[0][-top_n:][::-1]
    return data.iloc[recommendations]

# Получение рекомендаций
recommended_products = recommend_hybrid(user_profile, data[:100]['текст'].tolist())
recommended_products[['название', 'ключевые слова',  'цена', 'ссылка', 'код товара']]

ValueError: Expected 2D array, got 1D array instead:
array=[ 1.29374862e-01 -6.42155390e-03  2.00620532e-01 -3.17758858e-01
 -2.03787237e-01  2.61379898e-01  1.06578663e-01 -2.57226583e-02
  3.55661035e-01 -2.12697417e-01  3.01197678e-01  2.87725627e-01
 -1.85120299e-01 -3.68283093e-01 -2.78757423e-01 -1.66703761e-01
  5.60701601e-02  2.91499555e-01 -1.82715312e-01  4.16289717e-01
  1.09217405e-01  6.25826240e-01 -1.42488126e-02  4.18527797e-02
 -4.85604964e-02 -1.02858603e-01  7.92486295e-02  3.50846231e-01
 -2.44112208e-01  3.36753204e-02 -2.38462299e-01  3.22285920e-01
 -7.04105720e-02 -1.35366321e-01 -7.55567625e-02 -3.12760882e-02
 -1.57096654e-01  5.66343069e-01 -3.22307236e-02  5.51178217e-01
 -4.47533697e-01  5.69839180e-02  4.44624811e-01 -1.58008143e-01
  1.94815770e-02  3.16515148e-01 -1.07428350e-01  3.39996040e-01
  1.53516680e-01  1.80761963e-01 -5.21938987e-02 -3.04128945e-01
  5.64269945e-02 -2.63218105e-01 -9.45610628e-02  2.64355481e-01
  4.37157631e-01  2.63619244e-0

In [ ]:
data.to_csv('out.csv', index=False)

## **4. Ранжирование и фильтрация:**

* **Ранжирование:** Отсортируйте рекомендации по убыванию сходства или прогнозируемого рейтинга.
* **Фильтрация:** Примените фильтры по цене, категории, подкатегории и т.д., чтобы сузить список рекомендаций в соответствии с предпочтениями пользователя.



## **5.  Дополнительные функции:**

* **Персонализация:** Используйте информацию о прошлых взаимодействиях пользователя с системой для повышения релевантности рекомендаций.
* **Разнообразие:** Стремитесь к разнообразию в рекомендациях, избегая предлагать слишком похожие товары.
* **Объяснение:** Отобразите пользователю, почему ему были рекомендованы те или иные товары.